In [ ]:
#!pip install git+https://github.com/brianmanderson/PyESAPI

In [3]:
import pyesapi
import atexit
import SimpleITK as sitk
import os
import numpy as np

In [4]:
def get_direction(varian_image):
    x_direction = [varian_image.get_XDirection()[i] for i in range(3)]
    y_direction = [varian_image.get_YDirection()[i] for i in range(3)]
    z_direction = [varian_image.get_ZDirection()[i] for i in range(3)]
    direction = x_direction + y_direction + z_direction
    return tuple(direction)

def get_spacing(varian_image):
    return (varian_image.get_XRes(), varian_image.get_YRes(), varian_image.get_ZRes())

def get_origin(potential_image):
    return tuple([potential_image.get_Origin()[i] for i in range(3)])

In [20]:
def create_sitk_Image(varian_image):
    intercept = varian_image.VoxelToDisplayValue(0)
    image_array = varian_image.np_array_like().transpose()
    image_handle = sitk.GetImageFromArray(image_array + intercept)
    image_handle.SetSpacing(get_spacing(varian_image))
    image_handle.SetDirection(get_direction(varian_image))
    image_handle.SetOrigin(get_origin(varian_image))
    return image_handle

In [6]:
def create_sitk_Mask(varian_mask, varian_image):
    image_handle = sitk.GetImageFromArray(varian_mask.np_mask_like(varian_image).transpose())
    image_handle.SetSpacing(get_spacing(varian_image))
    image_handle.SetDirection(get_direction(varian_image))
    image_handle.SetOrigin(get_origin(varian_image))
    return image_handle

In [7]:
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose)

<bound method 'Dispose'>

In [8]:
#for pat_sum in app.PatientSummaries:
#    print(pat_sum.Id, pat_sum.LastName, pat_sum.FirstName)

In [9]:
fid = open(os.path.join('.', 'PatientID'))
line = fid.readline()
line = fid.readline() # Go to the second patient
fid.close()
MRN = line.strip('\n')

In [10]:
patient = app.OpenPatientById(MRN)

In [11]:
courses = [c.Id for c in patient.Courses]

In [12]:
plan = patient.CoursesLot(courses[1]).PlanSetupsLot(0)

In [13]:
type(plan.Dose)

VMS.TPS.Common.Model.API.PlanningItemDose

In [8]:
print(f'Plan Id: {plan.Id}')
print(f'Dose Per Fx: {plan.PrescribedDosePerFraction}')
print(f'Number of Fx: {plan.NumberOfFractions}')

Plan Id: 1 Abdomen
Dose Per Fx: 300.0 cGy
Number of Fx: 10


In [56]:
np.min(beams[0].Dose.np_array_like() == plan.Dose.np_array_like())

False

In [29]:
beams[0].Dose.get_ZRes()

2.5

In [ ]:
print(f'StructureID,TYPE,VOLUME')
for structure in plan.StructureSet.Structures:
    print(f'{structure.Id},{structure.DicomType},{structure.Volume:.2f}')

In [21]:
image_handle = create_sitk_Image(plan.StructureSet.Image)

In [19]:
plan.StructureSet.Image.VoxelToDisplayValue(0)

-1000.0

In [22]:
sitk.WriteImage(image_handle, r'K:\Image.nii.gz')

In [44]:
mask_handle = create_sitk_Mask(plan.StructureSet.StructuresLot('Liver'), plan.StructureSet.Image)

In [62]:
dir(plan.StructureSet.Image)

['ApprovalHistory',
 'CalculateDectProtonStoppingPowers',
 'ClearSerializationHistory',
 'Comment',
 'ContrastBolusAgentIngredientName',
 'CreateNewStructureSet',
 'CreationDateTime',
 'DicomToUser',
 'DisplayUnit',
 'EndSerialization',
 'Equals',
 'FOR',
 'Finalize',
 'GetHashCode',
 'GetImageProfile',
 'GetProtonStoppingPowerCurve',
 'GetSchema',
 'GetType',
 'GetVoxels',
 'HasUserOrigin',
 'HistoryDateTime',
 'HistoryUserDisplayName',
 'HistoryUserName',
 'Id',
 'ImageType',
 'ImagingOrientation',
 'IsProcessed',
 'Level',
 'MemberwiseClone',
 'Name',
 'Origin',
 'Overloads',
 'ReadXml',
 'ReferenceEquals',
 'Series',
 'StartSerialization',
 'ToString',
 'UID',
 'UserOrigin',
 'UserOriginComments',
 'UserToDicom',
 'VoxelToDisplayValue',
 'Window',
 'WriteXml',
 'XDirection',
 'XRes',
 'XSize',
 'YDirection',
 'YRes',
 'YSize',
 'ZDirection',
 'ZRes',
 'ZSize',
 '__call__',
 '__class__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '_

In [67]:
plan.StructureSet.Image.VoxelToDisplayValue()

TypeError: No method matches given arguments for VoxelToDisplayValue: ()

In [ ]:
sitk.WriteImage(mask_handle, r'K:\Mask.nii.gz')

In [45]:
dose_handle = create_sitk_Image(plan.Dose)

In [46]:
sitk.WriteImage(dose_handle, r'K:\Dose.nii.gz')

In [60]:
for i in range(len(beams)):
    if beams[i].Dose is None:
        continue
    beam_handle = create_sitk_Image(beams[i].Dose)
    sitk.WriteImage(dose_handle, r'K:\{}.nii.gz'.format(beams[i].Id))

In [ ]:
plan.StructureSet.StructuresLot('Liver').Volume # cc